# Notebook to design PLPs for ISS 


## What is this notebook doing?

This notebook takes you through the process of design of padlock probes for direct RNA detection using the Home Made dRNA protocol from the Nilsson lab. 

**As most of the code we write, we suggest you to run this in Linux. Surely `ClustalW2` and `cutadapt` can be run on Windows or Mac, but try this at your own risk and only if you vaguely know what you're doing**

The process works roughly as follows:

1) The user inputs a list of genes, a transcriptome and some other parameters

2) The `extract_and_align` function goes through the list of genes and extracts the corresponding mRNA sequences for all the isoforms from the transcriptome. All the mRNA sequences corresponding to a gene are aligned using ClustalW2 and the common regions are extracted. From these common regions, all the k-mers (whose length is specified by the user in the `plp_length` variable) that are suitable for binding a probe are extracted. This selection takes into account GC content and other variables determined by the RNA ligation chemistry. At this step you can also inspect the returned variables to figure out whether this step failed for some of your genes.

3) Of all the suitable k-mers (normally a large number per gene) only a subset is sampled from random non-overlapping positions on the mRNA molecule. The size of this subset is specified by the user in the variable `number_of_seqs`. The sequences in the subset will undergo the downstream step. **See note on computation time to optimise this number according to your needs**

4) The subsetted sequences will go a step of **specificicy check to remove off-targets**. Each sequence is searched back against the reference transcriptome, and any match that could potentially give spurious signal is excluded. This requires the user to introduce a mismatch tolerance parameter, for our current ligase we normally exclude any off-target with a mismatch up to 6nt. This parameter can be modified by the variable `mismatches`. We suggest not to change it and act conservatively. Often it's just impossible to design probes for closely related sequences (ie. recent paralogues) with the current detection chemistry.

5) At this point, the probe design pipeline might have failed for some genes, most likely because of a too low `number_of_seqs` value, which produced few sequences to map, or maybe because the mapped sequences are not unique enough in the transcriptome and pick off targets. You can inspect how your search went following the steps in the **"Check the amount of specific sequences for each gene"** section.

6) Very often, at this stage is advisable to perform a second run on the failed genes, trying to increase the `number_of_seqs` value. Check the paragraph "RUN2..."

7) Suitable and unique targets collected from both runs are finally merged into a single table. This table is then used for the design of probes, in the `Assign Gene to Barcode` section. Refer to that section for more specific instructions.



### Requirements to run the script

This script guides you through the design of a panel of padlock probes against a specified gene set. 

**Before you start**, make sure you have:

(1) A list of genes you want to design directRNA padlock probes for. 
This needs to be provided as a text file made of a single column of items starting with the header "Gene", and followed by a gene name per row. See attached examples in the Github folder

(2) The reference transcriptome of the species you are targeting in fasta format (.fasta/.fa/.fna) downloaded locally. 

**SEE IMPORTANT NOTE BELOW FOR THE FORMAT OF FASTA HEADERS**

(3) ClustalW2 downloaded on your computer, and correctly installed. See instructions for linux at the link below:

https://bioinformaticsreview.com/20210126/how-to-install-clustalw2-on-ubuntu/

(4) Cutadapt downloaded on your computer. This can be installed in the appropriate python environment by simply typing the following command: 

`pip install cutadapt` 

Please refer to the `cutadapt` manual in case of errors





## NOTE ON COMPUTATION TIME

The `map_sequences` function is very slow (it's basically a pattern matching at the whole transcriptome level). It runs by default as a multi-thread search, but it's still the slowest step in the pipeline, especially when working with large gene lists. This is the reason why we implemented a 2-step search based on 2 consecutive runs. You can of course work straight in searching a large number of `number_of_seqs` from the first run, but this will considerably slow down your search.

## IMPORTANT NOTE ABOUT FASTA HEADERS


The gene design pipeline should work for any species, provided you specify the correct gene names. 

However, the first function (`extract_and_align`) relies on a specific header format to extract the mRNA sequences based on the gene list provided. The function searches for the Gene names in the FASTA headers (line starting with >) using the format (<GENENAME>). 
    
If your reference transcriptome doesn't specify the gene name in this format, the script will fail extracting the sequences, so in that case you will need to reformat the FASTA headers of your transcriptome. There are a number of  tools and trick to reformat fasta headers, but the instructions will really depend on how your initial headers look like. Consult a bioinformatician if you're completely lost.

## Import packages

The first step in this script is to load the packages that the script will need. If anything fails in this step, it means most likely that you haven't installed the package yet. You can install packages with "pip install < whatever >" on the command line

In [1]:
import pandas as pd 
import Bio
from Bio import SeqIO
from Bio.SeqUtils import GC
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Align.Applications import ClustalwCommandline
import collections
from Bio import AlignIO
import random
import numpy as np
import sys
from PLP_directRNA_design import probedesign as plp

##  Reference parameters

Please specify in the following lines where your input csv, output path and accompaning software is located in your computer

`Path`: The folder where you want the output to be stored in.

`Genes`: csv containing the list of genes you want to design PLPs against. Must be a single column with a list of gene names, and the column must have `Gene` as a header.

`Pathclustal`: path where the program ClustalW2 is installed. Find it by searching ClustalW2 on your folder menu, opening the location of it and copying the path here

`ref`: reference transcriptome of the species you want to design probes for, in FASTA format (**please read once again the important note about FASTA headers**)

 `L_probe_library`:this is a file that links your gene list to ISS barcodes (LbarIDs) that will be used later for decoding. See the provided example file.
 
`LbarID`: Starting point of your barcoding attribution scheme (see below for further details)

In [2]:
path='C:/Users/sergio.salas/Documents/PhD/projects/gene_design/hower_example_5'
genes=pd.read_csv(r"C:\Users\sergio.salas\Documents\PhD\projects\gene_design\genelist2.csv",sep=',')
pathclustal=r"C:/ProgramData/Microsoft/Windows/Start Menu/Programs/ClustalW2/ClustalW2.lnk"
ref='C:/Users/sergio.salas/Documents/PhD/projects/gene_design/example/GRCh38_latest_rna.fna'
L_probe_library="C:/Users/sergio.salas/Documents/PhD/projects/gene_design/example/Lprobe_Ver2.csv"
LbarID=201 # 

# PARAMETERS RELATED WITH PLP DESIGN ITSELF

`number_of_seqs`: number of sequences (out of the ones fitting your parameters), that you want to map against the transcriptome.
The more you map, the slower it will be, but the higher chance you'll have to get specific sequences.
We recommend `number_of_seqs`=7-8, for a standard ISS experiment where you want 4-5 final PLPs

`final_designed`: maximum number of PLPs that you want to design out of the number included as selgenes.

`plp_length`: length of the part of the PLP hybridizing the RNA molecules.

`mismatches`: is the maximum allowed mismatches between the probe and the target sequence. Anything below or equal to `mismatches` will produce a positive hit. We recommend using 6, although 5 might be also okay if there are no seqs found to be specific with 6 nt. 

OPTIONAL: include `gc_max`, `gc_min`, set to 50 and 65 in the default script if you want to change it 

In [3]:
number_of_seqs= 10 # number of sequences mapped/PLP
final_designed=5 # number of PLPs designed at the end/gene
plp_length=30
mismatches = 6

# Extract and align sequences

The first step in the PLP design is to extract the sequnces for the designed genes from the reference transcriptome. Since many of the genes present more than one variant, the default script for PLP design will use only the common sequences

In [4]:
sequences,found_genes,notfound_genes=plp.extract_and_align(genes,ref,path,pathclustal) # optionally include gc_max, gc_min and plp_length to define parameters of your PLP

NameError: name 'genes' is not defined

From the previous function, we return `sequences`, which include all the potential sequences (not mapped yet) for all genes targeted. `found_genes` include the list of genes that we have been able to find k-mers fullfiling the PLP requirements. `not_found_genes` include the list of genes that we haven't been able to find/ we haven't been able to find any k-mer that fullfils the requirements. Those can be (1) too short genes or (2) genes with too many variants and no comon sequence among them

# Select a subsample of potential targets

We subsample the long list of potential targets/sequences to check for specificity. The returned object `selected_sequences` has the same format as sequences, but has only the amount of targets/gene specified in `number_of_seqs`

In [5]:
selected_sequences=plp.select_sequences(path,sequences,found_genes,number_of_seqs,subgroup=1)

SOX1
SOX3


For mapping the sequences, we define the following function to use `cutadapt` from the command line. Be aware that  to use the following function, you should first download and add cutadapt to the PATH or specify its path the function below. Refer to the instruction linked in the intro

In [2]:
def map_sequences(selected,subgroup=1):
    kmers =list(selected['Sequence'])
    transcriptome = (ref)
    seqlist = []
    hitlist = []
    lenlist = []
    s=0
    for sequence in kmers:
        s=s+1
        print ('Looking for sequence ('+str(s)+'/'+str(len(kmers))+'): '+sequence + ' allowing ' + str(mismatches) + ' mismatches')
        ########################################MODIFY CUTADAPT PATH IF NEEDED#############################
        output= !cutadapt -j 0 -a $sequence --overlap 30 --untrimmed-output /dev/null $transcriptome --no-indels -e $mismatches --action=none  
        n=0
        c2 = [line for line in output if line[0:1] == '>']
        print ('Found '+str(len(c2))+' hits')
        seqlist.append (sequence)
        hitlist.append (c2)
        lenlist.append (len(c2))
    expoutput = pd.DataFrame(list(zip(seqlist, hitlist, lenlist)),
                   columns =['sequence_with_Ns', 'hits', 'number_of_hits'])
    bcf_all=pd.concat([selected.reset_index(),expoutput],axis=1)
    bcf_all.to_csv(path+'mapped_sequences'+str(subgroup)+'.csv')
    return bcf_all

In [4]:
sel_mapped_seqs=map_sequences(selected_sequences,subgroup=1)

NameError: name 'selected_sequences' is not defined

# Check the amount of specific sequences for each gene

After mapping the sequences against the reference genome, we check how many genes have all PLPs that we asked for ( `genes_good_PLPs` ), how many have less than expected (`genes_too_low_PLPs`) and how many have none ( `genes_no_PLPs` ). If we add to this groups, the list of `notfound_genes` where we couldn't design any k-mer against, all the genes we tested should fall under one of this categories. This is:
* `notfound_genes`: genes either not found in the transcriptome OR without a common sequence in all variants/too short genes. We suggest to explore this genes further more to check if it's a naming problem or if it's a gene that is not suitable for ISS
* `genes_no_PLPs`: potential k-mers have been found for that gene, but none has proven to be specific. Try rerunning `plp.select_sequences` but with a larger `number_of_seqs`. If genes persist in this list, it can mean that they do not have specific k-mers suitable for ISS due to lack of specificity
* `genes_too_low_PLPs`: some potential k-mers have been found for this genes but they are not enough. Rerunning `plp.select_sequences` but with a larger `number_of_seqs` should really make it. Otherwise, consider ordering less PLPs for your gene of interest
* `genes_good_PLPs`: you have everything that you needed for those genes

In [45]:
bcf_all=sel_mapped_seqs
final_designed=final_designed
genes=genes
path=path
subgroup=1

In [54]:
specific_seqs1,genes_good_PLPs, genes_too_low_PLPs, genes_no_PLPs=plp.check_plps(sel_mapped_seqs,final_designed,genes,path,subgroup=1)

# RUN2 : looking for specific k-mers for genes missing (subgroup 2)

It's possible that you have group of genes that do not have enough specific k-mers in the first run. For these groups (present in `genes_too_low_PLPs` and `genes_no_PLPs` we will run another round of selection of potential k-mers to check their specificity. For this reason, we'll now modify the parameter `number_of_seqs` to be larger. This is considered as **`subgroup`=2** , and it's important to specify it in the functions. 

In [10]:
genes_to_redesign=list(genes_too_low_PLPs.index) # add genes_no_PLPs if needed to this list
number_of_seqs=10

In [11]:
selected_sequences2=plp.select_sequences(path,sequences,found_genes,number_of_seqs,subgroup=2)

In [12]:
sel_mapped_seqs2=map_sequences(selected_sequences2,subgroup=2)

Looking for sequence (1/3): ACTCCAAGGCTGTGTGTGCACCAGCCAGAG allowing 5 mismatches
Found 1 hits
Looking for sequence (2/3): ACTGCCTTACACCATGGACGCGTTCATGCT allowing 5 mismatches
Found 11 hits
Looking for sequence (3/3): CTCTACTGGCTGTATCTGGCATGAACGGTG allowing 5 mismatches
Found 2 hits


In [13]:
specific_seqs2,genes_good_PLPs2, genes_too_low_PLPs2, genes_no_PLPs2=plp.check_plps(sel_mapped_seqs2,final_designed,genes,path,subgroup=2)

# Combine the specific sequences from different subgroups

After we got the specific sequences for more than 1 run, we'd like to combine them to get a final list

In [14]:
final_seqs=pd.concat([specific_seqs1,specific_seqs2],axis=0)

In [15]:
specific_seqs_final,genes_good_PLPs_final, genes_too_low_PLPs_final, genes_no_PLPs_final=plp.check_plps(final_seqs,final_designed,genes,path,subgroup='final')

# Assign Gene to Barcode

 Once we have specific targets for all genes, we can build the padlock sequences. For this, we use our L_library_v2 file as reference. By modifying the parameter `how`, we can select how to assign genes to Lbar_IDs.
 
 * We can  select `how='start'` and then set `on` to the first ID to assign to your gene. The first gene will have the first ID, the second gene will have the first ID+1, and so on.
 * We can  select `how='end'` and then set `on` to the last ID to assign to your gene. Same as before, but starting from the end and going backwards.
 * We can also select `how='customized'` and provide as `on` the path to the csv with two columns, one for "Gene", and the other for "Lbar_ID"
 

In [16]:
customizedlib=r"C:\Users\sergio.salas\Documents\PhD\projects\gene_design\hower_example_5\assigned_gene_LID.csv"
probes=plp.build_plps(path,specific_seqs_final,L_probe_library,plp_length,how='customized',on=customizedlib)

I just processed 9 unique target sequences. I am done


In [17]:
probes

,sequence,Lbar_ID,AffyID,Gene,code
0,CTGACTTCGTTCCACACGTGCCTAAAGTGTACTACTGCGTCTATTT...,LbarID_0227,Afy16K_ID2081,GLI3,232311
1,ATGAAGACTGACCACACGTGCCTAAAGTGTACTACTGCGTCTATTT...,LbarID_0227,Afy16K_ID2081,GLI3,232311
2,CATGGTGTAAGGCAGAGGTCTTGACAATAAGGACGTGCGTCTATTT...,LbarID_0229,Afy16K_ID2086,MSI2,441111
3,ATGGTGTAAGGCAGTAGGTCTTGACAATAAGGACGTGCGTCTATTT...,LbarID_0229,Afy16K_ID2086,MSI2,441111
4,CTGAGCCTCATCTTGAGGTCGTTAAGAGGCAATACTGCGTCTATTT...,LbarID_0228,Afy16K_ID2085,NR2E1,111211
5,CGTTAGCTGAGCCTCAGGTCGTTAAGAGGCAATACTGCGTCTATTT...,LbarID_0228,Afy16K_ID2085,NR2E1,111211
